<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Backpropagation Practice

## *Data Science Unit 4 Sprint 2 Assignment 2*

Using TensorFlow Keras, Implement a 3 input, 4 node hidden-layer, 1 output node Multilayer Perceptron on the following dataset:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 0  | 0  | 1  | 0 |
| 0  | 1  | 1  | 1 |
| 1  | 0  | 1  | 1 |
| 0  | 1  | 0  | 1 |
| 1  | 0  | 0  | 1 |
| 1  | 1  | 1  | 0 |
| 0  | 0  | 0  | 0 |

If you look at the data you'll notice that the first two columns behave like an XOR gate while the last column is mostly just noise. Remember that creating an XOR gate was what the perceptron was criticized for not being able to learn.

This is your "Hello World!" of TensorFlow.

### Example TensorFlow Starter Code

```python 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
    Dense(3, activation='sigmoid', input_dim=2),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['acc'])

results = model.fit(X,y, epochs=100)

```

### Additional Written Tasks:
1. Investigate the various [loss functions](https://www.tensorflow.org/api_docs/python/tf/keras/losses). Which is best suited for the task at hand (predicting 1 / 0) and why? 

I would probably stick with the binary cross-entropy function. Most of the rest seem better suited to regression or multi-class classification. 

2. What is the difference between a loss function and a metric? Why might we need both in Keras? 

A loss function is the internal tool used to make changes to weights and biases to iteratively adapt to a set of training data. A metric is used to judge how well your model fits to the training data. Perhaps the metrics are there to give the data scientist something interesting to look at while the machine works.

3. Investigate the various [optimizers](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers). Stochastic Gradient Descent (`sgd`) is not the learning algorithm dejour anyone. Why is that? What do newer optimizers such as `adam` have to offer? 

It seems like Adam is an extended version of stochastic gradient descent that allows the algorithm to essentially take a peek in a bunch of different directions before deciding how far to go in a given direction.

In [2]:
import pandas as pd

df = pd.DataFrame(data=[[0,0,1,0],
                        [0,1,1,1],
                        [1,0,1,1],
                        [0,1,0,1],
                        [1,0,0,1],
                        [1,1,1,0],
                        [0,0,0,0]],
                  columns=['x1', 'x2', 'x3', 'y'],)

In [3]:
df

,x1,x2,x3,y
0,0,0,1,0
1,0,1,1,1
2,1,0,1,1
3,0,1,0,1
4,1,0,0,1
5,1,1,1,0
6,0,0,0,0


In [4]:
X = df[['x1','x2','x3']]
y = df[['y']]

In [16]:
import numpy as np
X = np.asarray(X)
y = np.asarray(y)

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
 
model = Sequential([
    Dense(3, activation='relu', input_dim=3),
    Dense(4, activation='relu'),
    Dense(1, activation='relu')
])
 
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
 
model.fit(X, y, epochs=700)

Train on 7 samples
Epoch 1/700
7/7 [==============================] - 0s 64ms/sample - loss: 4.6331 - acc: 0.5714
Epoch 2/700
7/7 [==============================] - 0s 429us/sample - loss: 4.6276 - acc: 0.5714
Epoch 3/700
7/7 [==============================] - 0s 434us/sample - loss: 4.6223 - acc: 0.5714
Epoch 4/700
7/7 [==============================] - 0s 286us/sample - loss: 4.6173 - acc: 0.5714
Epoch 5/700
7/7 [==============================] - 0s 571us/sample - loss: 4.6123 - acc: 0.5714
Epoch 6/700
7/7 [==============================] - 0s 857us/sample - loss: 4.6076 - acc: 0.5714
Epoch 7/700
7/7 [==============================] - 0s 571us/sample - loss: 4.6030 - acc: 0.5714
Epoch 8/700
7/7 [==============================] - 0s 429us/sample - loss: 4.5987 - acc: 0.5714
Epoch 9/700
7/7 [==============================] - 0s 429us/sample - loss: 4.5945 - acc: 0.5714
Epoch 10/700
7/7 [==============================] - 0s 428us/sample - loss: 4.5904 - acc: 0.5714
Epoch 11/700
7/7 [===

### Build a Tensor Keras Perceptron

Try to match the architecture we used on Monday - inputs nodes and one output node. Apply this architecture to the XOR-ish dataset above. 

After fitting your model answer these questions: 

Are you able to achieve the same results as a bigger architecture from the first part of the assignment? Why is this disparity the case? What properties of the XOR dataset would cause this disparity? 

So I haven't looked at this closely, but I think I'm actually getting the same results faster with the smaller architecture. The XOR dataset, as a logic gate with noise injected into it, may just be a poor fit for our usual expectations of how neural networks scale on larger naturalistic datasets. 

Now extrapolate this behavior on a much larger dataset in terms of features. What kind of architecture decisions could we make to avoid the problems the XOR dataset presents at scale? 

We could elect to make our neural networks more or less complicated in proportion to the complexity of meaning we expect them to find in the data.

*Note:* The bias term is baked in by default in the Dense layer.

In [26]:
model = Sequential([
    Dense(3, activation='relu', input_dim=3),
    Dense(1, activation='relu')
])
 
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
 
results = model.fit(X,y, epochs=100)

Train on 7 samples
Epoch 1/100
7/7 [==============================] - 0s 56ms/sample - loss: 4.5842 - acc: 0.5714
Epoch 2/100
7/7 [==============================] - 0s 428us/sample - loss: 2.1468 - acc: 0.5714
Epoch 3/100
7/7 [==============================] - 0s 284us/sample - loss: 1.9840 - acc: 0.5714
Epoch 4/100
7/7 [==============================] - 0s 571us/sample - loss: 1.8706 - acc: 0.5714
Epoch 5/100
7/7 [==============================] - 0s 286us/sample - loss: 1.7876 - acc: 0.5714
Epoch 6/100
7/7 [==============================] - 0s 290us/sample - loss: 1.7233 - acc: 0.5714
Epoch 7/100
7/7 [==============================] - 0s 425us/sample - loss: 1.6711 - acc: 0.5714
Epoch 8/100
7/7 [==============================] - 0s 432us/sample - loss: 1.6276 - acc: 0.5714
Epoch 9/100
7/7 [==============================] - 0s 429us/sample - loss: 1.5903 - acc: 0.5714
Epoch 10/100
7/7 [==============================] - 0s 429us/sample - loss: 1.5577 - acc: 0.5714
Epoch 11/100
7/7 [===

## Try building/training a more complex MLP on a bigger dataset.

Use TensorFlow Keras & the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) to build the canonical handwriting digit recognizer and see what kind of accuracy you can achieve. 

If you need inspiration, the Internet is chalk-full of tutorials, but I want you to see how far you can get on your own first. I've linked to the original MNIST dataset above but it will probably be easier to download data through a neural network library. If you reference outside resources make sure you understand every line of code that you're using from other sources, and share with your fellow students helpful resources that you find.


### Parts
1. Gathering & Transforming the Data
2. Making MNIST a Binary Problem
3. Estimating your Neural Network (the part you focus on)

### Gathering the Data 

`keras` has a handy method to pull the mnist dataset for you. You'll notice that each observation is a 28x28 arrary which represents an image. Although most Neural Network frameworks can handle higher dimensional data, that is more overhead than necessary for us. We need to flatten the image to one long row which will be 784 values (28X28). Basically, you will be appending each row to one another to make on really long row. 

In [27]:
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [28]:
# input image dimensions
img_rows, img_cols = 28, 28

In [29]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 1s 0us/step


In [32]:
x_train = x_train.reshape(x_train.shape[0], img_rows * img_cols)
x_test = x_test.reshape(x_test.shape[0], img_rows * img_cols)

# Normalize Our Data
x_train = x_train / 255
x_test = x_test / 255

In [33]:
# Now the data should be in a format you're more familiar with
x_train.shape

(60000, 784)

### Making MNIST a Binary Problem 
MNIST is multiclass classification problem; however we haven't covered all the necessary techniques to handle this yet. You would need to one-hot encode the target, use a different loss metric, and use softmax activations for the last layer. This is all stuff we'll cover later this week, but let us simplify the problem for now: Zero or all else.

In [34]:
import numpy as np

y_temp = np.zeros(y_train.shape)
y_temp[np.where(y_train == 0.0)[0]] = 1
y_train = y_temp

y_temp = np.zeros(y_test.shape)
y_temp[np.where(y_test == 0.0)[0]] = 1
y_test = y_temp

In [35]:
# A Nice Binary target for ya to work with
y_train

array([0., 1., 0., ..., 0., 0., 0.])

### Estimating Your `net

In [56]:
model = Sequential([
    Dense(784, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='relu')
])
 
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
 
model.fit(x_train, y_train, epochs=5)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 12s 195us/sample - loss: 0.0661 - acc: 0.9837
Epoch 2/5
60000/60000 [==============================] - 11s 187us/sample - loss: 0.0458 - acc: 0.9913
Epoch 3/5
60000/60000 [==============================] - 11s 190us/sample - loss: 0.0877 - acc: 0.9865
Epoch 4/5
60000/60000 [==============================] - 11s 189us/sample - loss: 0.0425 - acc: 0.9917
Epoch 5/5
60000/60000 [==============================] - 11s 186us/sample - loss: 0.0375 - acc: 0.9925


In [41]:
y_pred = model.predict(x_test)

In [55]:
from tensorflow.keras.metrics import binary_accuracy

binary_accuracy(y_test, y_pred)

<tf.Tensor: id=111316, shape=(10000,), dtype=float32, numpy=array([0.902, 0.902, 0.902, ..., 0.902, 0.902, 0.902], dtype=float32)>

If I'm reading this right, test accuracy right around 90%. Not terrible???

In [58]:
# Okay, now testing the second iteration of the net, with another hidden layer

model = Sequential([
    Dense(784, activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='relu')
])
 
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
 
model.fit(x_train, y_train, epochs=5)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 12s 205us/sample - loss: 0.0733 - acc: 0.9810
Epoch 2/5
60000/60000 [==============================] - 12s 193us/sample - loss: 0.0498 - acc: 0.9901
Epoch 3/5
60000/60000 [==============================] - 11s 191us/sample - loss: 0.0837 - acc: 0.9854
Epoch 4/5
60000/60000 [==============================] - 11s 191us/sample - loss: 0.0420 - acc: 0.9916
Epoch 5/5
60000/60000 [==============================] - 12s 194us/sample - loss: 0.0355 - acc: 0.9927


In [59]:
y_pred = model.predict(x_test)
binary_accuracy(y_test, y_pred)

<tf.Tensor: id=170290, shape=(10000,), dtype=float32, numpy=array([0.902, 0.902, 0.902, ..., 0.902, 0.902, 0.902], dtype=float32)>

## Stretch Goals: 

- Make MNIST a multiclass problem using cross entropy & soft-max
- Implement Cross Validation model evaluation on your MNIST implementation 
- Research different [Gradient Descent Based Optimizers](https://keras.io/optimizers/)
 - [Siraj Raval the evolution of gradient descent](https://www.youtube.com/watch?v=nhqo0u1a6fw)
- Build a housing price estimation model using a neural network. How does its accuracy compare with the regression models that we fit earlier on in class?